# Question 4.1

In [23]:
import json
import pprint
import pandas as pd
import csv
import os
import math
import matplotlib.pyplot as plt
import seaborn as sns

In [10]:
def q4():
    years = [2015, 2016, 2017, 2018]
    
    columns = ['eventIdx', 'game_id', 'Game Seconds', 'Game Period', 'X-Coordinate', 'Y-Coordinate',
               'Shot Distance', 'Shot Angle', 'Shot Type', 'Was Net Empty', 'Last Event Type', 'Last X-Coordinate',
               'Last Y-Coordinate', 'Time from Last Event (seconds)', 'Distance from Last Event', 'Is Rebound',
               'Change in Shot Angle', 'Speed', 'Is Goal']
    
    data = []
    
    def get_distance_angle(shoot_left, x_coor, y_coor):
        if shoot_left == True:
            distance = math.sqrt((89 + x_coor) ** 2 + (y_coor) ** 2)
            if x_coor == -89:
                if y_coor > 0:
                    angle = 0
                elif y_coor < 0:
                    angle = 180
                elif y_coor == 0:
                    angle = 90
            else:
                if y_coor > 0:
                    angle = math.degrees(math.atan((89 + x_coor) / y_coor))
                elif y_coor < 0:
                    angle = 180 - abs(math.degrees(math.atan((89 + x_coor) / y_coor)))
                elif y_coor == 0:
                    angle = 90
                    
        elif shoot_left == False:
            distance = math.sqrt((89 - x_coor) ** 2 + (y_coor) ** 2)
            if x_coor == 89:
                if y_coor > 0:
                    angle = 180
                elif y_coor < 0:
                    angle = 0
                elif y_coor == 0:
                    angle = 90
            else:
                if y_coor > 0:
                    angle = 180 - abs(math.degrees(math.atan((89 - x_coor) / y_coor)))
                elif y_coor < 0:
                    angle = abs(math.degrees(math.atan((89 - x_coor) / y_coor)))
                elif y_coor == 0:
                    angle = 90
                
        return distance, angle
    
    for year in years:
        count_jsons = 0
        for file in os.listdir(f'../../m2_JSON_data/regular_season/{year}'):
            if file[-5:] == '.json':
                count_jsons += 1
                # Opening JSON file
                f = open(f'../../m2_JSON_data/regular_season/{year}/{file}')

                game_id = int(file[:-5])

                loaded_json = json.load(f)

                all_plays = loaded_json['liveData']['plays']['allPlays']

                home_team = loaded_json['gameData']['teams']['home'].get('name')

                home_side = 'NA'
                if len(loaded_json['liveData']['linescore']['periods']) > 0:
                    home_side = loaded_json['liveData']['linescore']['periods'][0]['home'].get('rinkSide')
                    
                angle = 0
                
                for i in range(len(all_plays)):
                    event = all_plays[i]['result']['event']
                    if event == 'Shot' or event == 'Goal':
                        
                        # Q4.1 ################################################################################
                        
                        game_period = all_plays[i]['about']['period']
                        
                        period_time_mins, period_time_secs = all_plays[i]['about']['periodTime'].split(':')
                        game_seconds = ((int(game_period) - 1) * 1200) + (int(period_time_mins) * 60 + int(period_time_secs))
                        
                        team_name = all_plays[i]['team']['name']

                        if home_team == team_name:
                            home_or_away = 'Home'
                        else:
                            home_or_away = 'Away'
                        
                        eventIdx = all_plays[i]['about']['eventIdx']
                            
                        x_coor = 'NA'
                        y_coor = 'NA'
                        if 'coordinates' in all_plays[i]:
                            x_coor = all_plays[i]['coordinates'].get('x')
                            y_coor = all_plays[i]['coordinates'].get('y')
                    
                        x_coor = int(float(x_coor)) if bool(x_coor) and x_coor != 'NA' else False
                        y_coor = int(float(y_coor)) if bool(y_coor) and y_coor != 'NA' else False
                        
                        prev_angle = angle

                        if x_coor and y_coor:
                            if home_side == 'right':                    
                                if home_or_away == 'Home':
                                    if int(game_period) % 2 == 1:
                                        # shoot left
                                        distance, angle = get_distance_angle(True, x_coor, y_coor)

                                    elif int(game_period) % 2 == 0:
                                        # shoot right
                                        distance, angle = get_distance_angle(False, x_coor, y_coor)
                                else:
                                    if int(game_period) % 2 == 1:
                                        # shoot right
                                        distance, angle = get_distance_angle(False, x_coor, y_coor)

                                    elif int(game_period) % 2 == 0:
                                        # shoot left
                                        distance, angle = get_distance_angle(True, x_coor, y_coor)

                            elif home_side == 'left':
                                if home_or_away == 'Home':
                                    if int(game_period) % 2 == 1:
                                        # shoot right
                                        distance, angle = get_distance_angle(False, x_coor, y_coor)

                                    elif int(game_period) % 2 == 0:
                                        # shoot left
                                        distance, angle = get_distance_angle(True, x_coor, y_coor)
                                        
                                else:
                                    if int(game_period) % 2 == 1:
                                        # shoot left
                                        distance, angle = get_distance_angle(True, x_coor, y_coor)

                                    elif int(game_period) % 2 == 0:
                                        # shoot right
                                        distance, angle = get_distance_angle(False, x_coor, y_coor)
                            
                        
                        
                        shot_type = 'NA'
                        if 'secondaryType' in all_plays[i]['result']:
                            shot_type = all_plays[i]['result']['secondaryType']
                        
                        is_net_empty = False
                        if event == 'Goal':
                            if 'emptyNet' in all_plays[i]['result']:
                                is_net_empty = all_plays[i]['result']['emptyNet']
                                
                        # Q4.2 ################################################################################
                        
                        last_event = all_plays[i - 1]['result']['event']
                        
                        last_x_coor = 'NA'
                        last_y_coor = 'NA'
                        if 'coordinates' in all_plays[i - 1]:
                            last_x_coor = all_plays[i - 1]['coordinates'].get('x')
                            last_y_coor = all_plays[i - 1]['coordinates'].get('y')
                    
                        last_x_coor = int(float(last_x_coor)) if bool(last_x_coor) and last_x_coor != 'NA' else False
                        last_y_coor = int(float(last_y_coor)) if bool(last_y_coor) and last_y_coor != 'NA' else False
                        
                        last_game_period = all_plays[i - 1]['about']['period']
                        
                        last_period_time_mins, last_period_time_secs = all_plays[i - 1]['about']['periodTime'].split(':')
                        last_event_game_seconds = ((int(last_game_period) - 1) * 1200) + (int(last_period_time_mins) * 60 + int(last_period_time_secs))
                        time_from_last_event = game_seconds - last_event_game_seconds
                        
                        distance_from_last_event = math.sqrt(((x_coor - last_x_coor) ** 2) + ((y_coor - last_y_coor) ** 2))
                        
                        # Q4.3 ################################################################################
                        
                        is_rebound = True if last_event == 'Shot' and game_period == all_plays[i - 1]['about']['period'] else False
                        
                        change_in_shot_angle = angle - prev_angle if is_rebound else 0
                        
                        speed = distance_from_last_event / time_from_last_event if time_from_last_event != 0 else 0
                        
                        #######################################################################################

                        row_data = [eventIdx, game_id, game_seconds, game_period, x_coor, y_coor, 
                                    distance, angle, shot_type, int(is_net_empty), last_event, last_x_coor,
                                    last_y_coor, time_from_last_event, distance_from_last_event, is_rebound, 
                                    change_in_shot_angle, speed, int(event == 'Goal')]

                        data.append(row_data)
                        
        print('year', year)
        print('count_jsons', count_jsons)

    # create file path if it doesn't exist
    filename = f'../../m2_CSV_data/all_data_q4.csv'
    dirname = os.path.dirname(filename)
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    # writing to csv file 
    with open(filename, 'w+') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(columns) 

        # writing the data rows 
        csvwriter.writerows(data)    

In [11]:
q4()

year 2015
count_jsons 1230
year 2016
count_jsons 1230
year 2017
count_jsons 1271
year 2018
count_jsons 1271


In [31]:
def q4_categorical():
    # Shot Type, Last Event Type
    years = [2015, 2016, 2017, 2018]
    
    columns = ['eventIdx', 'game_id', 'Game Seconds', 'Game Period', 'X-Coordinate', 'Y-Coordinate',
               'Shot Distance', 'Shot Angle', 'Shot Type', 'Was Net Empty', 'Last Event Type', 'Last X-Coordinate',
               'Last Y-Coordinate', 'Time from Last Event (seconds)', 'Distance from Last Event', 'Is Rebound',
               'Change in Shot Angle', 'Speed', 'Is Goal']
    
    data = []
    
    def get_distance_angle(shoot_left, x_coor, y_coor):
        if shoot_left == True:
            distance = math.sqrt((89 + x_coor) ** 2 + (y_coor) ** 2)
            if x_coor == -89:
                if y_coor > 0:
                    angle = 0
                elif y_coor < 0:
                    angle = 180
                elif y_coor == 0:
                    angle = 90
            else:
                if y_coor > 0:
                    angle = math.degrees(math.atan((89 + x_coor) / y_coor))
                elif y_coor < 0:
                    angle = 180 - abs(math.degrees(math.atan((89 + x_coor) / y_coor)))
                elif y_coor == 0:
                    angle = 90
                    
        elif shoot_left == False:
            distance = math.sqrt((89 - x_coor) ** 2 + (y_coor) ** 2)
            if x_coor == 89:
                if y_coor > 0:
                    angle = 180
                elif y_coor < 0:
                    angle = 0
                elif y_coor == 0:
                    angle = 90
            else:
                if y_coor > 0:
                    angle = 180 - abs(math.degrees(math.atan((89 - x_coor) / y_coor)))
                elif y_coor < 0:
                    angle = abs(math.degrees(math.atan((89 - x_coor) / y_coor)))
                elif y_coor == 0:
                    angle = 90
                
        return distance, angle
    
    for year in years:
        count_jsons = 0
        for file in os.listdir(f'../../m2_JSON_data/regular_season/{year}'):
            if file[-5:] == '.json':
                count_jsons += 1
                # Opening JSON file
                f = open(f'../../m2_JSON_data/regular_season/{year}/{file}')

                game_id = int(file[:-5])

                loaded_json = json.load(f)

                all_plays = loaded_json['liveData']['plays']['allPlays']

                home_team = loaded_json['gameData']['teams']['home'].get('name')

                home_side = 'NA'
                if len(loaded_json['liveData']['linescore']['periods']) > 0:
                    home_side = loaded_json['liveData']['linescore']['periods'][0]['home'].get('rinkSide')
                    
                angle = 0
                
                for i in range(len(all_plays)):
                    event = all_plays[i]['result']['event']
                    if event == 'Shot' or event == 'Goal':
                        
                        # Q4.1 ################################################################################
                        
                        game_period = all_plays[i]['about']['period']
                        
                        period_time_mins, period_time_secs = all_plays[i]['about']['periodTime'].split(':')
                        game_seconds = ((int(game_period) - 1) * 1200) + (int(period_time_mins) * 60 + int(period_time_secs))
                        
                        team_name = all_plays[i]['team']['name']

                        if home_team == team_name:
                            home_or_away = 'Home'
                        else:
                            home_or_away = 'Away'
                        
                        eventIdx = all_plays[i]['about']['eventIdx']
                            
                        x_coor = 'NA'
                        y_coor = 'NA'
                        if 'coordinates' in all_plays[i]:
                            x_coor = all_plays[i]['coordinates'].get('x')
                            y_coor = all_plays[i]['coordinates'].get('y')
                    
                        x_coor = int(float(x_coor)) if bool(x_coor) and x_coor != 'NA' else False
                        y_coor = int(float(y_coor)) if bool(y_coor) and y_coor != 'NA' else False
                        
                        prev_angle = angle

                        if x_coor and y_coor:
                            if home_side == 'right':                    
                                if home_or_away == 'Home':
                                    if int(game_period) % 2 == 1:
                                        # shoot left
                                        distance, angle = get_distance_angle(True, x_coor, y_coor)

                                    elif int(game_period) % 2 == 0:
                                        # shoot right
                                        distance, angle = get_distance_angle(False, x_coor, y_coor)
                                else:
                                    if int(game_period) % 2 == 1:
                                        # shoot right
                                        distance, angle = get_distance_angle(False, x_coor, y_coor)

                                    elif int(game_period) % 2 == 0:
                                        # shoot left
                                        distance, angle = get_distance_angle(True, x_coor, y_coor)

                            elif home_side == 'left':
                                if home_or_away == 'Home':
                                    if int(game_period) % 2 == 1:
                                        # shoot right
                                        distance, angle = get_distance_angle(False, x_coor, y_coor)

                                    elif int(game_period) % 2 == 0:
                                        # shoot left
                                        distance, angle = get_distance_angle(True, x_coor, y_coor)
                                        
                                else:
                                    if int(game_period) % 2 == 1:
                                        # shoot left
                                        distance, angle = get_distance_angle(True, x_coor, y_coor)

                                    elif int(game_period) % 2 == 0:
                                        # shoot right
                                        distance, angle = get_distance_angle(False, x_coor, y_coor)
                            
                        
                        
                        shot_type = 'NA'
                        if 'secondaryType' in all_plays[i]['result']:
                            shot_type = all_plays[i]['result']['secondaryType']
                        
                        is_net_empty = False
                        if event == 'Goal':
                            if 'emptyNet' in all_plays[i]['result']:
                                is_net_empty = all_plays[i]['result']['emptyNet']
                                
                        # Q4.2 ################################################################################
                        
                        last_event = all_plays[i - 1]['result']['event']
                        
                        last_x_coor = 'NA'
                        last_y_coor = 'NA'
                        if 'coordinates' in all_plays[i - 1]:
                            last_x_coor = all_plays[i - 1]['coordinates'].get('x')
                            last_y_coor = all_plays[i - 1]['coordinates'].get('y')
                    
                        last_x_coor = int(float(last_x_coor)) if bool(last_x_coor) and last_x_coor != 'NA' else False
                        last_y_coor = int(float(last_y_coor)) if bool(last_y_coor) and last_y_coor != 'NA' else False
                        
                        last_game_period = all_plays[i - 1]['about']['period']
                        
                        last_period_time_mins, last_period_time_secs = all_plays[i - 1]['about']['periodTime'].split(':')
                        last_event_game_seconds = ((int(last_game_period) - 1) * 1200) + (int(last_period_time_mins) * 60 + int(last_period_time_secs))
                        time_from_last_event = game_seconds - last_event_game_seconds
                        
                        distance_from_last_event = math.sqrt(((x_coor - last_x_coor) ** 2) + ((y_coor - last_y_coor) ** 2))
                        
                        # Q4.3 ################################################################################
                        
                        is_rebound = True if last_event == 'Shot' and game_period == all_plays[i - 1]['about']['period'] else False
                        
                        change_in_shot_angle = angle - prev_angle if is_rebound else 0
                        
                        speed = distance_from_last_event / time_from_last_event if time_from_last_event != 0 else 0
                        
                        #######################################################################################

                        row_data = [eventIdx, game_id, game_seconds, game_period, x_coor, y_coor, 
                                    distance, angle, shot_type, int(is_net_empty), last_event, last_x_coor,
                                    last_y_coor, time_from_last_event, distance_from_last_event, is_rebound, 
                                    change_in_shot_angle, speed, int(event == 'Goal')]

                        if type(x_coor) == int and type(y_coor) == int and type(last_x_coor) == int and type(last_y_coor) == int:
                            data.append(row_data)
                        
        print('year', year)
        print('count_jsons', count_jsons)

    # data['Shot Type'] = data['Shot Type'].astype('category')
    # data['Last Event Type'] = data['Last Event Type'].astype('category')
    df = pd.DataFrame.from_records(data)
    
    unique_shot_types = df.iloc[:, 8].unique()
    print(unique_shot_types)
    df.iloc[:, 8].replace(to_replace=unique_shot_types,
           value=list(range(len(unique_shot_types))),
           inplace=True)
    
    unique_last_event_types = df.iloc[:, 10].unique()
    print(unique_last_event_types)
    df.iloc[:, 10].replace(to_replace=unique_last_event_types,
           value=list(range(len(unique_last_event_types))),
           inplace=True)
    
    print(df.dtypes)
        
    data = df.values.tolist()
    
    # create file path if it doesn't exist
    filename = f'../../m2_CSV_data/all_data_q4_categorical.csv'
    dirname = os.path.dirname(filename)
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    # writing to csv file 
    with open(filename, 'w+') as csvfile: 
        # creating a csv writer object 
        csvwriter = csv.writer(csvfile) 

        # writing the fields 
        csvwriter.writerow(columns) 

        # writing the data rows 
        csvwriter.writerows(data)    

In [32]:
q4_categorical()

year 2015
count_jsons 1230
year 2016
count_jsons 1230
year 2017
count_jsons 1271
year 2018
count_jsons 1271
['Wrist Shot' 'Backhand' 'Slap Shot' 'Snap Shot' 'Tip-In' 'Deflected'
 'Wrap-around' 'NA']
['Hit' 'Faceoff' 'Takeaway' 'Blocked Shot' 'Giveaway' 'Shot' 'Missed Shot'
 'Goal' 'Penalty']
0       int64
1       int64
2       int64
3       int64
4       int64
5       int64
6     float64
7     float64
8       int64
9       int64
10      int64
11      int64
12      int64
13      int64
14    float64
15       bool
16    float64
17    float64
18      int64
dtype: object
